# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
%matplotlib inline
# import numpy and pandas
import numpy as np
import pandas as pd


# Challenge 1 - Analysis of Variance

In this part of the lesson, we will perform an analysis of variance to determine whether the factors in our model create a significant difference in the group means. We will be examining a dataset of FIFA players. We'll start by loading the data using the code in the cell below.

In [2]:
# Run this code:

fifa = pd.read_csv('fifa.csv')

Let's examine the dataset by looking at the `head`.

In [3]:
# Your code here:

fifa.head()

,Name,Age,Nationality,Overall,Potential,Club,Value,Preferred Foot,Position,Weak Foot,Acceleration,SprintSpeed,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties
0,L. Messi,31,Argentina,94,94,FC Barcelona,€110.5M,Left,RF,4.0,91.0,86.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0
1,Cristiano Ronaldo,33,Portugal,94,94,Juventus,€77M,Right,ST,4.0,89.0,91.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0
2,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,€118.5M,Right,LW,5.0,94.0,90.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0
3,De Gea,27,Spain,91,93,Manchester United,€72M,Right,GK,3.0,57.0,58.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0
4,K. De Bruyne,27,Belgium,91,92,Manchester City,€102M,Right,RCM,5.0,78.0,76.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0


Player's values are expressed in millions of euros. We would like this column to be numeric. Therefore, let's create a numeric value column. Do this by stripping all non-numeric characters from each cell. Assign this new data to `ValueNumeric`. There is no need to multiply the value to be expressed in millions. 

In [4]:
# Your code here:

fifa['ValueNumeric'] = fifa['Value'].replace(to_replace='€0', value='€0N').apply(lambda i: float(i[1:-1]) if i[-1:] == 'M' else float(i[1:-1])/1000)

In [5]:
fifa.head()

,Name,Age,Nationality,Overall,Potential,Club,Value,Preferred Foot,Position,Weak Foot,...,SprintSpeed,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,ValueNumeric
0,L. Messi,31,Argentina,94,94,FC Barcelona,€110.5M,Left,RF,4.0,...,86.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,110.5
1,Cristiano Ronaldo,33,Portugal,94,94,Juventus,€77M,Right,ST,4.0,...,91.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,77.0
2,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,€118.5M,Right,LW,5.0,...,90.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,118.5
3,De Gea,27,Spain,91,93,Manchester United,€72M,Right,GK,3.0,...,58.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,72.0
4,K. De Bruyne,27,Belgium,91,92,Manchester City,€102M,Right,RCM,5.0,...,76.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,102.0


#### We'd like to determine whether a player's preffered foot and position have an impact on their value. 

Using the `statsmodels` library, we are able to produce an ANOVA table without munging our data. Create an ANOVA table with value as a function of position and preferred foot. Recall that pivoting is performed by the `C` function.

Hint: For columns that have a space in their name, it is best to refer to the column using the dataframe (For example: for column `A`, we will use `df['A']`).

In [6]:
# Your code here:
from scipy.stats import f_oneway

# table = fifa.pivot_table(values='ValueNumeric', index=['Position'], columns=['Preferred Foot'], aggfunc=np.sum)

fifa = fifa[fifa['Position'].isna() == False]
fifa = fifa[fifa['Preferred Foot'].isna() == False]

fifa.reset_index(drop = True, inplace = True)


In [27]:
table = pd.DataFrame()

fifa['Position_Foot'] = [fifa['Position'][i]+'_'+fifa['Preferred Foot'][i] for i in range(len(fifa['Position']))]

arr_pf_unique = fifa.groupby(['Position_Foot'])['ValueNumeric'].count().sort_values(ascending = False).index

fifa.groupby(['Position_Foot'])['ValueNumeric'].value_counts()

for i in arr_pf_unique:
    table[i] = fifa[fifa['Position_Foot'] == i]['ValueNumeric'].reset_index(drop=True)
    

# table.reset_index(drop = True)

# table[table['ST_Right'].isna() == True]



,ST_Right,GK_Right,CB_Right,RB_Right,LB_Left,CM_Right,RM_Right,CDM_Right,CAM_Right,LM_Right,...,RAM_Right,LF_Right,LWB_Right,LAM_Right,RF_Right,RAM_Left,LAM_Left,RF_Left,RWB_Left,LF_Left
0,77.0,72.0,44.0,35.0,43.0,45.5,81.0,51.5,73.5,59.0,...,29.5,93.0,13.5,1.6,45.0,32.5,69.5,110.50,0.400,89.000
1,77.0,68.0,30.0,31.5,38.0,50.5,27.5,59.5,53.0,52.0,...,2.8,21.5,5.5,1.2,11.5,7.0,17.5,6.50,0.375,9.000
2,83.5,58.0,28.5,24.5,36.5,38.0,15.5,44.0,45.0,43.5,...,5.5,22.5,1.9,1.2,4.2,3.8,7.0,6.50,0.375,0.425
3,64.5,38.0,9.0,26.5,38.0,32.0,30.5,18.0,26.0,45.0,...,2.8,7.5,6.0,2.5,3.3,2.2,2.8,1.50,0.375,NaN
4,64.5,30.0,15.5,20.0,21.5,34.0,31.0,34.0,35.5,39.0,...,2.9,5.0,2.7,1.7,3.9,1.8,5.5,0.22,NaN,NaN


In [29]:
[target[idxs] for idxs in

p_value = f_oneway(*table)
p_value

ValueError: could not convert string to float: 'ST_Right'

What is your conclusion from this ANOVA?

In [6]:
# Your conclusions here:



After looking at a model of both preffered foot and position, we decide to create an ANOVA table for nationality. Create an ANOVA table for numeric value as a function of nationality.

In [8]:
# Your code here:



What is your conclusion from this ANOVA?

# Challenge 2 - Linear Regression

Our goal with using linear regression is to create a mathematical model that will enable us to predict the outcome of one variable using one or more additional independent variables.

We'll start by ensuring there are no missing values. Examine all variables for all missing values. If there are missing values in a row, remove the entire row.

In [9]:
# Your code here:



Using the FIFA dataset, in the cell below, create a linear model predicting value using stamina and sprint speed. create the model using `statsmodels`. Print the model summary.

Hint: remember to add an intercept to the model using the `add_constant` function.

In [10]:
# Your code here:



Report your findings from the model summary. In particular, report about the model as a whole using the F-test and how much variation is predicted by the model using the r squared.

In [11]:
# Your conclusions here:



Next, create a second regression model predicting value using potential. Create the model using `statsmodels` and print the model summary. Remember to add a constant term.

In [12]:
# Your code here:



Report your findings from the model summary. In particular, report about the model as a whole using the F-test and how much variation is predicted by the model using the r squared.

In [13]:
# Your conclusions here:



Plot a scatter plot of value vs. potential. Do you see a linear relationship?

In [14]:
# Your code here:

